In [28]:
!pip install transformers sentence-transformers chromadb pypdf langchain


In [35]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/kaggle/input/hscbangla1st/HSC26-Bangla1st-Paper.pdf")
pages = loader.load_and_split()

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
chunks = splitter.split_documents(pages)


In [36]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embedding = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
db = Chroma.from_documents(chunks, embedding=embedding, persist_directory="./vectordb")
retriever = db.as_retriever()


In [37]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="deepset/xlm-roberta-base-squad2",
    tokenizer="deepset/xlm-roberta-base-squad2"
)


Device set to use cuda:0


In [38]:
def custom_rag_qa(query, retriever, qa_pipeline):
    docs = retriever.get_relevant_documents(query)
    context = " ".join([doc.page_content for doc in docs[:2]])
    result = qa_pipeline(question=query, context=context)
    return result['answer']


In [39]:
questions = [
    "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
    "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?",
    "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
]

for q in questions:
    print(f"Q: {q}")
    print("A:", custom_rag_qa(q, retriever, qa_pipeline))
    print("-" * 50)


Q: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
A:  যলাভী
--------------------------------------------------
Q: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
A:  যলাভী মানর্সকতাএব্ং
--------------------------------------------------
Q: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
A: 
44
--------------------------------------------------
